### JSON parsing


In [ ]:
import json
from urllib.request import urlopen


def getCountry(ipAddress: str):
    response = urlopen('http://freegeoip.net/json/' +
                       ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get('country_code')


print(getCountry('50.78.253.58'))

In [5]:
import json
from urllib.request import urlopen


def getCountry(ipAddress: str) -> (str | None):
    response = urlopen(f'https://ipapi.co/{ipAddress}/json').read().decode(
        'utf-8')
    responseJson = json.loads(response)
    return responseJson.get('country_name')


print(getCountry('50.78.253.58'))

United States


In [ ]:
import json

jsonString = '{"arrayOfNums":[{"number":0},{"number":1},{"number":2}],"arrayOfFruits":[{"fruit":"apple"},{"fruit":"banana"},{"fruit":"pear"}]}'
jsonObj = json.loads(jsonString)

print(jsonObj.get('arrayOfNums'))
print(jsonObj.get('arrayOfNums')[1])
print(
    jsonObj.get('arrayOfNums')[1].get('number') +
    jsonObj.get('arrayOfNums')[2].get('number'))
print(jsonObj.get('arrayOfFruits')[2].get('fruit'))

### Combining APIs with other sources of data


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, ResultSet
from typing import Any
import json
import datetime
import random
import re

random.seed(datetime.datetime.now())


def getLinks(articleUrl: str) -> ResultSet[Any]:
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')

    return bs.find('div', {
        'id': 'bodyContent'
    }).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))


def getHistoryIPs(pageUrl: str) -> set:
    # This is the format of revision history pages:
    # http://en.wikipedia.org/w/index.php?title=Title_in_URL&action=history
    pageUrl = pageUrl.replace('/wiki/', '')
    historyUrl = 'http://en.wikipedia.org/w/index.php?title={}&action=history'.format(
        pageUrl)

    print('history url is: {}'.format(historyUrl))
    html = urlopen(historyUrl)
    bs = BeautifulSoup(html, 'html.parser')
    # finds only links whose class is "mw-anonuserlink" and
    # have IP addresses instead of usernames
    ipAddresses = bs.findAll('a', {'class': 'mw-anonuserlink'})
    addressList = set()
    
    for ipAddress in ipAddresses:
        addressList.add(ipAddress.get_text())
        
    return addressList


def getCountry(ipAddress: str) -> (str | None):
    try:
        response = urlopen(f'https://ipapi.co/{ipAddress}/json').read().decode(
            'utf-8')
    except:
        return None
    responseJson = json.loads(response)
    return responseJson.get('country_name')


links = getLinks('/wiki/Python_(programming_language)')
while (len(links) > 0):
    for link in links:
        print('-' * 20)
        historyIPs = getHistoryIPs(link.attrs['href'])
        for historyIP in historyIPs:
            print(historyIP)
            
    newLink = links[random.randint(0, len(links) - 1)].attrs['href']
    links = getLinks(newLink)
    